In [1]:
import dlisio
import pandas as pd
import numpy as np
import cv2

In [2]:
#FMS
def open_fms(filename:str) -> pd.DataFrame:
    f, *f_tail = dlisio.load(filename)
    if len(f_tail): logging.warning('There are more logical files in tail')
    
    frame =f.object('FRAME','1B').curves()
    
    df = pd.DataFrame(frame)
    df["TDEP"]=df["TDEP"]* 0.00254
    df.drop(["FRAMENO"], axis=1, inplace=True)
    
    #df = normalize_values(df)
    
    
   # cols_list = [['TDEP','BA11', 'BA12', 'BA13', 'BA14', 'BA15', 'BA16', 'BA17', 'BA18', 
   #          'BA21','BA22', 'BA23', 'BA24', 'BA25', 'BA26', 'BA27', 'BA28'],
   #         ['TDEP','BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 
   #          'BB21','BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28'],
   #         ['TDEP','BC11', 'BC12', 'BC13', 'BC14', 'BC15', 'BC16', 'BC17', 'BC18', 
   #          'BC21','BC22', 'BC23', 'BC24', 'BC25', 'BC26', 'BC27', 'BC28']]

    
    #pads = [df[cols].copy() for cols in cols_list]
    #pads_norm = [normalize_values(pad) for pad in pads]
    #pads_concatenated = [concatenate_normalized(pad) for pad in pads_norm]  
    return normalize_values(df)

def normalize_values(data:pd.DataFrame) -> pd.DataFrame:
    TDEP= data['TDEP'].copy()
    data.drop(['TDEP'], axis=1, inplace=True)
    MAX_FMS_VALUE = 100
    data = (data-data.min())/(MAX_FMS_VALUE-data.min())*255
    data['TDEP'] = TDEP        
    return data

def core_info(filename:str)-> pd.DataFrame:
    data = pd.read_csv(filename)
    data['max_depth'] = data['min_depth']+data['core_v_pixel']/20000
    print((data['core_v_pixel']-data['core_v_pixel']%512)/512)
    return data

def FMS_to_core(data:pd.DataFrame):
    FMS_list= []
    for index, interval in core_info.iterrows():
        min_depth = core_info.min_depth
        max_depth = core_info.max_depth
        FMS_coredepth = (open_fms[(open_fms['TDEP']>=min_depth) & (open_fms['TDEP']<=max_depth)])
        FMS_nparray=create_FMS_image(FMS_coredepth)
        FMS_list.append(FMS_nparray)
    return FMS_list

def create_FMS_image(FMS_coredepth):
    FMS123 = FMS_list(["BA11","BB11","BC11","BA12","BB12","BC12",
                       "BA13","BB13","BC13","BA14","BB14","BC14",
                       "BA15","BB15","BC15","BA16","BB16","BC16",
                       "BA17","BB17","BC17","BA18","BB18","BC18",
                       "BA21","BB21","BC21","BA22","BB22","BC22",
                       "BA23","BB23","BC23","BA24","BB24","BC24",
                       "BA25","BB25","BC25","BA26","BB26","BC26",
                       "BA27","BB27","BC27","BA28","BB28","BC28"]) 
    FMS123_array = FMS123.values
    FMS123_image = FMS123_array.reshape(len(FMS123),16,3).astype(np.uint8)
    print(FMS123_image.shape)
    return FMS123_image

def resize_FMS_image(FMS123_image, dim, interpolation=cv2.INTER_LANCZOS4):
    width = 512
    height = core_info.core_v_pixel
    dim = (width, height)
    FMS = cv2.resize(FMS123_image, dim, interpolation)
    return FMS

def save_FMS_image(img):
    for i in range (FMS):
        for j in range (FMS):
            path = '../raw_data/194_FMS_Images/1196A/'
            filename = str(core_info.leg)+str('-')+str(core_info.site)+str(core_info.hole)+str('-')
            +str(core_info.core)+str(core_info.t)+str('-')+str(core.info_sc)+str('_')
            +str(core_info.core_min)+str(core_info.core_max)
            frmt = 'tif'
            cv2.imwrite('{}.{}'.format(filename, format), img)
            print (f'Successfully created {FMS} images')
        return images
    
    sliced_FMS = [slice_FMS(images,filename) for images in img]
    return sliced_image

def slice_FMS(img, overlap=0.5):
    slice_dim = 512
    stride = int((1-overlap)*slice_dim)
    FMS_images = int(img.shape[0]/stride)-1
    
    FMS_slices= []
    for i in range(FMS_images):
        for j in range (FMS_images):
            slice = img[i:i+slice_dim, j:j+slice_dim]
            FMS_slices.append(slice)
            name = f'{filename}_splitted'
            frmt = 'jpg'
            cv2.imwrite('{}_{}.{}'.format(name, i, frmt), slice)
            print(f'Succesfully created {FMS_images} images')
            return FMS_slices

In [3]:
#CORE
def crop_slice_image(filename:str):
    '''crops the core images width to 512pixel from mid +-256pixels'''
    img = cv2.imread(filename)
    img_h, img_w, _ = img.shape
    w1, h1 = int(img_w/2)-256, int(img_h)
    cropped_image = img[w1:w1+512, 0:h1]
    if cropped_image.shape[0]<512:
        return [] # return an empty list if the image is not big enough. Otherwise continues the code below...

    sliced_image = [slice_image(cropped_image,filename) for cropped_image in img]
    return sliced_image

def slice_image(img, filename, overlap=0.5):
    '''slice the core images to 512x512 pixel and overlaps images by 50%.
    Notice that I take the cropped image as my input to the function'''
    slice_dim = 512
    stride = int((1-overlap)*slice_dim) #The stride in pixel is simply the overlap time the pixel dimentions
    nb_images = int(img.shape[0]/stride)-1
    #The total number of image is equal to how many full strides you can fit minus 1!
    
    # Now let's do a loop between 0 and the total number of images, and save the images:
    slices = []
    
    for i in range(nb_images):
        for j in range (nb_images):
            slice = img[i:i+slice_dim, j:j+slice_dim]
            slices.append(slice)
            count = 0
            name = f'{filename}_splitted'
            frmt = 'jpg'
            cv2.imwrite('{}_{}.{}'.format(name, i, frmt), slice)
            count +=1
        
    print(f'Succesfully created {nb_images} images')
    return slices

In [4]:
open_fms('../raw_data/MESTB _009_1B.dlis')

,BA11,BA12,BA13,BA14,BA15,BA16,BA17,BA18,BA21,BA22,...,BD18,BD21,BD22,BD23,BD24,BD25,BD26,BD27,BD28,TDEP
0,1.479395,0.603696,2.672543,1.748999,1.117807,1.101806,0.923172,1.962298,2.076046,0.806138,...,1.931255,0.954731,0.744779,1.399806,1.053719,1.085582,1.679640,2.225671,6.147987,818.23560
1,1.479395,0.603696,2.672543,1.748999,1.117807,1.101806,0.923172,1.962298,2.076046,0.806138,...,1.931255,0.954731,0.744779,1.399806,1.053719,1.085582,1.679640,2.225671,6.147987,818.23306
2,1.479395,0.603696,2.672543,1.748999,1.117807,1.101806,0.923172,1.962298,2.076046,0.806138,...,1.931255,0.954731,0.744779,1.399806,1.053719,1.085582,1.679640,2.225671,6.147987,818.23052
3,1.479395,0.603696,2.672543,1.748999,1.117807,1.101806,0.923172,1.962298,2.076046,0.806138,...,1.931255,0.954731,0.744779,1.399806,1.053719,1.085582,1.679640,2.225671,6.147987,818.22798
4,1.479395,0.603696,2.672543,1.748999,1.117807,1.101806,0.923172,1.962298,2.076046,0.806138,...,1.931255,0.954731,0.744779,1.399806,1.053719,1.085582,1.679640,2.225671,6.147987,818.22544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173035,71.451920,51.773224,43.758270,38.119953,35.839912,34.651180,36.208176,40.072838,57.389290,47.636547,...,20.699097,25.802538,23.038504,21.616468,20.206612,19.658331,20.275366,21.279882,23.883148,378.72670
173036,71.451920,51.773224,43.758270,38.119953,35.839912,34.651180,36.208176,40.072838,57.389290,47.636547,...,20.699097,25.802538,23.038504,21.616468,20.206612,19.658331,20.275366,21.279882,23.883148,378.72416
173037,71.451920,51.773224,43.758270,38.119953,35.839912,34.651180,36.208176,40.072838,57.389290,47.636547,...,20.699097,25.802538,23.038504,21.616468,20.206612,19.658331,20.275366,21.279882,23.883148,378.72162
173038,71.451920,51.773224,43.758270,38.119953,35.839912,34.651180,36.208176,40.072838,57.389290,47.636547,...,20.699097,25.802538,23.038504,21.616468,20.206612,19.658331,20.275366,21.279882,23.883148,378.71908


In [5]:
core_info('../raw_data/1196A_coreinfo.csv')

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
372    1.0
373    1.0
374    3.0
375    7.0
376    3.0
Name: core_v_pixel, Length: 377, dtype: float64


,leg,site,hole,core,t,sc,min_depth,max_depth,core_min,core_max,core_v_pixel,coreimage_path
0,194,1196,A,10,R,1,399.575,399.61025,7.5,11.0,705,../raw_data/194_Core_Images/1196A/ 194-1196A-1...
1,194,1196,A,10,R,1,399.620,399.65985,12.0,16.0,797,../raw_data/194_Core_Images/1196A/ 194-1196A-1...
2,194,1196,A,10,R,1,399.720,399.75025,22.0,25.0,605,../raw_data/194_Core_Images/1196A/ 194-1196A-1...
3,194,1196,A,10,R,1,399.770,399.79695,27.0,30.0,539,../raw_data/194_Core_Images/1196A/ 194-1196A-1...
4,194,1196,A,10,R,1,399.850,399.88005,35.0,38.0,601,../raw_data/194_Core_Images/1196A/ 194-1196A-1...
...,...,...,...,...,...,...,...,...,...,...,...,...
372,194,1196,A,52,R,2,805.305,805.35505,13.5,18.5,1001,../raw_data/194_Core_Images/1196A/ 194-1196A-5...
373,194,1196,A,52,R,2,805.375,805.40130,20.5,23.0,526,../raw_data/194_Core_Images/1196A/ 194-1196A-5...
374,194,1196,A,52,R,2,805.425,805.50495,25.5,33.5,1599,../raw_data/194_Core_Images/1196A/ 194-1196A-5...
375,194,1196,A,52,R,2,805.515,805.70045,34.5,53.0,3709,../raw_data/194_Core_Images/1196A/ 194-1196A-5...


In [6]:
filename = '../raw_data/ 194-1196A-10R-1_7.511.tif'
slices = crop_slice_image(filename)

Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully created 1 images
Succesfully cr

In [ ]:
#CORE 2ND VERSION
def crop_slice_image(filename:str):
    '''crops the core images width to 512pixel from mid +-256pixels'''
    img = cv2.imread(filename)
    img_h, img_w, _ = img.shape
    w1, h1 = int(img_w/2)-256, int(img_h)
    cropped_image = img[w1:w1+512, 0:h1]
    if cropped_image.shape[0]<512:
        return [] # return an empty list if the image is not big enough. Otherwise continues the code below...

    sliced_image = [slice_images(cropped_image,filename) for cropped_image in img]
    return sliced_image

def slice_images(cropped_image, filename):
    img = cropped_image
    slice_width = 512
    slice_height = 512
    return img
    
def start_points(size, slice_size, overlap=0):
    points = [0]
    stride = int(slice_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + slice_size >= size:
            points.append(size - slice_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points

    X_points = start_points(img_w, slice_width, 0.5)
    Y_points = start_points(img_h, slice_height, 0.5)

    count = 0
    name = 'splitted'
    frmt = 'jpg'

    for i in Y_points:
        for j in X_points:
            slice = img[i:i+slice_height, j:j+slice_width]
            cv2.imwrite('{}_{}.{}'.format(name, count, frmt), slice)
            count += 1
            
    print(f'Succesfully created {sliced_images} images')
    return slices